In [2]:
import os
import re
import joblib
import ir_datasets
import nltk
from tqdm import tqdm
from pymongo import MongoClient
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import TfidfVectorizer

# تحميل موارد NLTK
nltk.download("punkt")
nltk.download("wordnet")
nltk.download("omw-1.4")

def custom_tokenizer(text):
    return text.split()

lemmatizer = WordNetLemmatizer()

def tokenize(text):
    text = text.lower()
    tokens = re.findall(r'\b\w+\b', text)
    return tokens

def lemmatize_tokens(tokens):
    return [lemmatizer.lemmatize(token) for token in tokens]

def clean_text(text):
    tokens = tokenize(text)
    lemmas = lemmatize_tokens(tokens)
    return " ".join(lemmas)

# تحميل الاستعلامات من BEIR
dataset = ir_datasets.load("beir/quora/test")

query_ids = []
original_texts = []
clean_texts = []

print("🧼 تنظيف الاستعلامات...")
for query in tqdm(dataset.queries_iter()):
    cleaned = clean_text(query.text)
    if cleaned.strip():
        query_ids.append(query.query_id)
        original_texts.append(query.text)
        clean_texts.append(cleaned)

# تحميل vectorizer الخاص بالوثائق
doc_vectorizer_path = r"C:\Users\Reem Darawsheh\Desktop\PythonProject\PythonProject\Data Representation\TF-IDF\beir\quora\test\doc\tfidf_data.joblib"
doc_vectorizer = joblib.load(doc_vectorizer_path)["vectorizer"]

# تحويل الاستعلامات إلى تمثيل TF-IDF
print("🔢 تحويل الاستعلامات باستخدام vectorizer الخاص بالوثائق...")
query_tfidf_matrix = doc_vectorizer.transform(clean_texts)

# حفظ النتائج إلى ملف joblib
output_dir = r"C:\Users\Reem Darawsheh\Desktop\PythonProject\PythonProject\Query Processing\tfidf"
os.makedirs(output_dir, exist_ok=True)
output_path = os.path.join(output_dir, "queries_tfidf_reencoded.joblib")

joblib.dump({
    "query_ids": query_ids,
    "original_texts": original_texts,
    "clean_texts": clean_texts,
    "query_tfidf_matrix": query_tfidf_matrix,
    "vectorizer": doc_vectorizer
}, output_path)

print(f"✅ تم حفظ تمثيل الاستعلامات المتوافق في: {output_path}")

# 🧭 تخزين في MongoDB
print("📦 تخزين الاستعلامات في MongoDB...")

client = MongoClient("mongodb://localhost:27017/")
db = client["ir_project"]
collection = db["queries_quora_test_tfidf_reencoded"]

# حذف المجموعة القديمة إن وُجدت
collection.delete_many({})

docs = []
for i in range(len(query_ids)):
    vec = query_tfidf_matrix[i].tocoo()  # تحويل إلى صيغة COO لسهولة التعامل
    doc = {
        "query_id": query_ids[i],
        "original_text": original_texts[i],
        "clean_text": clean_texts[i],
        "tfidf_indices": vec.col.tolist(),
        "tfidf_values": vec.data.tolist()
    }
    docs.append(doc)

collection.insert_many(docs)
print(f"✅ تم تخزين {len(docs)} استعلام في MongoDB داخل المجموعة: {collection.name}")


[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Azzam\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Azzam\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\Azzam\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


🧼 تنظيف الاستعلامات...


10000it [00:01, 8490.86it/s]


🔢 تحويل الاستعلامات باستخدام vectorizer الخاص بالوثائق...
✅ تم حفظ تمثيل الاستعلامات المتوافق في: C:\Users\Azzam\PycharmProjects\PythonProject\Query Processing\tfidf\queries_tfidf_reencoded.joblib
📦 تخزين الاستعلامات في MongoDB...
✅ تم تخزين 10000 استعلام في MongoDB داخل المجموعة: queries_quora_test_tfidf_reencoded


In [6]:
import os
import re
import joblib
import ir_datasets
import nltk
from tqdm import tqdm
from pymongo import MongoClient
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import TfidfVectorizer

# تحميل موارد NLTK
nltk.download("punkt")
nltk.download("wordnet")
nltk.download("omw-1.4")

def custom_tokenizer(text):
    return text.split()

lemmatizer = WordNetLemmatizer()

def tokenize(text):
    text = text.lower()
    tokens = re.findall(r'\b\w+\b', text)
    return tokens

def lemmatize_tokens(tokens):
    return [lemmatizer.lemmatize(token) for token in tokens]

def clean_text(text):
    tokens = tokenize(text)
    lemmas = lemmatize_tokens(tokens)
    return " ".join(lemmas)

# تحميل الاستعلامات من BEIR
dataset = ir_datasets.load("antique/test/non-offensive")

query_ids = []
original_texts = []
clean_texts = []

print("🧼 تنظيف الاستعلامات...")
for query in tqdm(dataset.queries_iter()):
    cleaned = clean_text(query.text)
    if cleaned.strip():
        query_ids.append(query.query_id)
        original_texts.append(query.text)
        clean_texts.append(cleaned)

# تحميل vectorizer الخاص بالوثائق
doc_vectorizer_path = r"C:\Users\Reem Darawsheh\Desktop\PythonProject\PythonProject\Data Representation\TF-IDF\antique\train\doc\tfidf_data.joblib"
doc_vectorizer = joblib.load(doc_vectorizer_path)["vectorizer"]

# تحويل الاستعلامات إلى تمثيل TF-IDF
print("🔢 تحويل الاستعلامات باستخدام vectorizer الخاص بالوثائق...")
query_tfidf_matrix = doc_vectorizer.transform(clean_texts)

# حفظ النتائج إلى ملف joblib
output_dir = r"C:\Users\Reem Darawsheh\Desktop\PythonProject\PythonProject\Query Processing\tfidf"
os.makedirs(output_dir, exist_ok=True)
output_path = os.path.join(output_dir, "queries_tfidf_antique.joblib")

joblib.dump({
    "query_ids": query_ids,
    "original_texts": original_texts,
    "clean_texts": clean_texts,
    "query_tfidf_matrix": query_tfidf_matrix,
    "vectorizer": doc_vectorizer
}, output_path)

print(f"✅ تم حفظ تمثيل الاستعلامات المتوافق في: {output_path}")
#
# # 🧭 تخزين في MongoDB
# print("📦 تخزين الاستعلامات في MongoDB...")
#
# client = MongoClient("mongodb://localhost:27017/")
# db = client["ir_project"]
# collection = db["queries_antique_train_tfidf_reencoded"]
#
# # حذف المجموعة القديمة إن وُجدت
# collection.delete_many({})
#
# docs = []
# for i in range(len(query_ids)):
#     vec = query_tfidf_matrix[i].tocoo()  # تحويل إلى صيغة COO لسهولة التعامل
#     doc = {
#         "query_id": query_ids[i],
#         "original_text": original_texts[i],
#         "clean_text": clean_texts[i],
#         "tfidf_indices": vec.col.tolist(),
#         "tfidf_values": vec.data.tolist()
#     }
#     docs.append(doc)
#
# collection.insert_many(docs)
# print(f"✅ تم تخزين {len(docs)} استعلام في MongoDB داخل المجموعة: {collection.name}")


[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Azzam\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Azzam\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\Azzam\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


🧼 تنظيف الاستعلامات...


[INFO] Please confirm you agree to the authors' data usage agreement found at <https://ciir.cs.umass.edu/downloads/Antique/readme.txt>
[INFO] [starting] https://ciir.cs.umass.edu/downloads/Antique/test-queries-blacklist.txt
0it [00:03, ?it/s]
                  ass.edu/downloads/Antique/test-queries-blacklist.txt: 0.0%| 0.00/184 [00:00<?, ?B/s]
                                                                                                   [INFO] [finished] https://ciir.cs.umass.edu/downloads/Antique/test-queries-blacklist.txt: [00:00] [184B] [183kB/s]
0it [00:03, ?it/s]
https://ciir.cs.umass.edu/downloads/Antique/test-queries-blacklist.txt: [00:00] [184B] [30.7kB/s]
176it [00:03, 51.07it/s]                                                                         


🔢 تحويل الاستعلامات باستخدام vectorizer الخاص بالوثائق...
✅ تم حفظ تمثيل الاستعلامات المتوافق في: C:\Users\Azzam\PycharmProjects\PythonProject\Query Processing\tfidf\queries_tfidf_antique.joblib
📦 تخزين الاستعلامات في MongoDB...
✅ تم تخزين 176 استعلام في MongoDB داخل المجموعة: queries_antique_train_tfidf_reencoded


In [2]:
import joblib
data = joblib.load(r"C:\Users\User\Desktop\PythonProject\PythonProject\Query Processing\tfidf\queries_tfidf_antique.joblib")
print(len(data["query_ids"]))
print(data["query_ids"][:10])


2426
['3097310', '3910705', '237390', '2247892', '1078492', '782453', '3198658', '1907320', '10895', '992730']


In [3]:
import joblib

bert_data = joblib.load(r"C:\Users\User\Desktop\PythonProject\PythonProject\Query Processing\Bertquery\antique\train\query_embeddings\bert_query_embeddings.joblib")
tfidf_data = joblib.load(r"C:\Users\User\Desktop\PythonProject\PythonProject\Query Processing\tfidf\queries_tfidf_antique.joblib")

bert_ids = set(bert_data["query_ids"])
tfidf_ids = set(tfidf_data["query_ids"])

common_ids = bert_ids & tfidf_ids
print("✅ الاستعلامات المشتركة:", len(common_ids))
print("📦 عدد استعلامات BERT:", len(bert_ids))
print("📦 عدد استعلامات TF-IDF:", len(tfidf_ids))
print("❌ الاستعلامات غير الموجودة في BERT:", len(tfidf_ids - bert_ids))
print("❌ الاستعلامات غير الموجودة في TF-IDF:", len(bert_ids - tfidf_ids))


✅ الاستعلامات المشتركة: 2426
📦 عدد استعلامات BERT: 2426
📦 عدد استعلامات TF-IDF: 2426
❌ الاستعلامات غير الموجودة في BERT: 0
❌ الاستعلامات غير الموجودة في TF-IDF: 0
